In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re

#Data handling
from sklearn.model_selection import train_test_split

# Models 
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier


# Normalizer
from sklearn.preprocessing import MinMaxScaler

#Pickling
from six.moves import cPickle as pickle

#Metrics
from sklearn.metrics import average_precision_score, recall_score, precision_score, f1_score

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline

x_train = []
y_train = []
x_test = []
y_test = []

In [4]:
def get_gender(text):
    text = text[len(text)-4]
    if('F' in text):
        return '0'
    elif('M' in text):
        return '1'
    else:
        return '-1'
    
def get_word_segments(align_file):
    segments = []
    with open(align_file,'r') as openFile:
        read_lines = openFile.readlines()
        lines = read_lines[1:len(read_lines)-1]
        for line in lines:
            splitted = line.split()
            segments.append({'sf': splitted[0], 'ef':splitted[1],'seg_scr':splitted[2],'word':splitted[3]})
    return segments

def filter_word_count(data, n_count):
    return data[list(map(lambda x: len(x.split(' ')) >= n_count,data['text']))]

def remove_empty_patterns(data,patterns):
    empty_patterns = [k for k, v in patterns.items() if len(v) < 1]
    patterns = { k:v for k, v in patterns.items() if len(v) >= 1 }
    data = filter(lambda x: x[1]['index'] not in empty_patterns ,data.iterrows())
    data = pd.DataFrame.from_items(data).T
    return data,patterns

def load_data(word_count, emotional_mapping):
    # full = generate_IEMOCAP_df()
    data = pd.read_csv('data/IEMOCAP_sentences_votebased.csv',index_col=0)
    data['emotion_code'] = data['emotion'].map( emotional_mapping ).astype(int)
    # Take away fear, surprise,disgust, xxx and others. Not enough data
    data = data[data.emotion_code < 4]
    # Clean Transcripts
    data['text'] = data['text'].apply(clean_text)
    # Filter Word Count
    data = filter_word_count(data, word_count)
#     data,patterns = remove_empty_patterns(data,patterns)
    return data

def clean_text(text):
    punct_str = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~«»“…‘”\t'
    for p in punct_str:
        text = text.replace(p,' ')
    text = re.sub(' +', ' ', text)
    return text.lower().strip()

In [5]:
import basic_models



In [6]:
emotional_mapping = {'ang': 0, 'sad': 1, 'hap': 2, 'neu': 3,'fru': 4,'exc': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data = load_data(3,emotional_mapping)

In [7]:
data

,index,start_time,end_time,text,wav_path,alignment_path,emotion,valence,arousal,dominance,gender,emotion_code
2,Ses01F_impro01_F002,14.8872,18.0175,is there a problem,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,2.5,2.5,F,3
5,Ses01F_impro01_F005,27.4600,31.4900,well what's the problem let me change it,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,3.5,2.0,F,3
12,Ses01F_impro01_F012,85.2700,88.0200,that's out of control,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,ang,2.0,3.5,3.5,F,0
14,Ses01F_impro01_F014,97.8900,102.9600,clearly you know do you have like a supervisor...,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.5,3.5,3.5,F,3
27,Ses01F_impro01_M011,87.1500,94.3900,i don't understand why this is so complicated ...,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,ang,2.0,4.0,4.0,M,0
29,Ses01F_impro01_M013,101.8400,107.8700,yeah do you want to see my supervisor huh yeah...,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,ang,2.0,4.5,4.5,M,0
30,Ses01F_impro02_F000,7.2688,16.6000,did you get the mail so you saw my letter,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,sad,2.5,2.0,1.5,F,1
31,Ses01F_impro02_F001,19.8500,22.7800,yeah i know,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,sad,2.0,1.5,1.5,F,1
32,Ses01F_impro02_F002,26.5659,32.2895,i don't know i put in that request too they di...,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,sad,2.0,1.5,1.5,F,1
33,Ses01F_impro02_F003,32.3075,37.9600,i guess you know everybody has to make sacrifices,data/IEMOCAP_full_release/Session1/sentences/w...,data/IEMOCAP_full_release/Session1/sentences/F...,neu,2.0,2.0,2.0,F,3


In [8]:
# def get_sample(data,emotional_mapping,emotion):
#     emotion_code = emotional_mapping[emotion]
#     emotion_sample = data[data.emotion_code == emotion_code]
#     sample_size = len(emotion_sample)
#     oth_emotion_sample = data[data.emotion_code != emotion_code].sample(n=sample_size)
#     sample = pd.concat([oth_emotion_sample,emotion_sample]).sample(frac=1)
#     y = map_for_emotion(sample.emotion_code,emotional_mapping,'ang')
#     return sample,y

def get_mfcc_feature(filename,n_mfcc):
    y, sr = librosa.load(filename)
    #IPython.display.Audio(data=y,rate=sr)
    mfcc = librosa.feature.mfcc(y=y,sr=sr, n_mfcc=n_mfcc)
    print("Finished "+ filename)
    return np.mean(mfcc.T,axis=0)

def get_rmse(filename):
    y, sr = librosa.load(filename)
    rmse = librosa.feature.rmse(y, frame_length=200)
    print("Finished "+ filename)
    return rmse[0]

def get_zcr(filename):
    y,sr = librosa.load(filename)
    zcr = librosa.feature.zero_crossing_rate(y  + 0.0100) # 0.0200 background noise reduction
    print("Finished "+ filename)
    return zcr[0] 

def get_spectrogram(filename,n_fft = 2048):
    y,sr = librosa.load(filename)
    Y = librosa.stft(y, n_fft=n_fft)
    Ymag = librosa.amplitude_to_db(np.absolute(Y))
    print('Finished '+ filename)
    return np.mean(Ymag.T,axis=0)

def get_melspectrogram(filename,n_fft = 4096):
    y,sr = librosa.load(filename)
    Y = librosa.feature.melspectrogram(y, n_fft=n_fft)
    Ymag = librosa.power_to_db(Y)
    print('Finished '+ filename)
    return np.mean(Ymag,axis=1)

def get_cqt(filename,n_fft = 4096):
    y,sr = librosa.load(filename)
    C = librosa.cqt(y,sr=sr)
    Clog = librosa.amplitude_to_db(np.abs(C))
    print('Finished '+ filename)
    return np.mean(Clog,axis=1)

def get_chroma(filename):
    y,sr = librosa.load(filename)
    chromagram = librosa.feature.chroma_cqt(y,sr=sr)
    print('Finished ' + filename)
    return np.mean(chromagram,axis=1)

def get_spectral_centroid(filename):
    y, sr = librosa.load(filename)
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
    print("Finished "+ filename)
    return spectral_centroids

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis,feature_range=(0, 1))

def play_audio(filename):
    y, sr = librosa.load(filename)
    return ipd.Audio(data=y,rate=sr)

def get_confusion(y_pred, y_true):
    tp = []
    fp =[]
    fn =[]
    tn = []
    for i in range(len(y_pred)):
        if(y_pred[i] == y_true[i]):
            if y_pred[i] == 1:
                tp.append(True)
                tn.append(False)
            else:
                tn.append(True)
                tp.append(False)
            fp.append(False)
            fn.append(False)
        else:
            if(y_pred[i] == 1):
                fp.append(True)
                fn.append(False)
            else:
                fp.append(False)
                fn.append(True)
            tn.append(False)
            tp.append(False)
            
    return tp,tn,fp,fn

def scale_features(x_train, x_test):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_x_train = scaler.fit_transform(x_train)
#     print(scaled_x_train.min(axis=0))
#     print(scaled_x_train.max(axis=0))
    scaled_x_test = scaler.fit_transform(x_test)
    return scaled_x_train,scaled_x_test

def pad_data(x_train, x_test):
    x_train = pd.DataFrame(x_train).fillna(0)
    x_test = pd.DataFrame(x_test).fillna(0)
    print(x_train.shape, x_test.shape)
    if(x_train.shape[1] > x_test.shape[1]):
        diff = x_train.shape[1] - x_test.shape[1]
        for i in range(diff):
            x_test[x_test.shape[1] + i] = 0
    else:
        diff = x_test.shape[1] - x_train.shape[1]
        for i in range(diff):
            x_train[x_train.shape[1] + i] = 0
    print(x_train.shape, x_test.shape)
    return x_train, x_test

def combine_features(data1,data2):
    return pd.concat([data1, data2],axis=1)


In [3]:
def load_dataset(data, emotion, compute = False ):
    if(compute):
        x,y = get_sample(data,emotional_mapping,'ang')
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        try:
            with open('pickles/'+emotion+'/base_datasets.pickle','wb') as f:
                save = {
                    'X_train' : X_train,
                    'X_test' : X_test,
                    'y_train': y_train,
                    'y_test' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to anger base datasets pickle: ', e)
        return X_train,X_test,y_train,y_test
    else:
        try:
            with open('pickles/'+emotion+'/base_datasets.pickle','rb') as f:
                save = pickle.load(f)
                X_train = save['X_train']
                X_test = save['X_test']
                y_train = save['y_train']
                y_test = save['y_test']
                del save
                return X_train,X_test,y_train,y_test
        except Exception as e:
            print('Error loading base datasets pickle: ', e)

def load_mfcc(emotion, X_train, X_test,y_train,y_test, compute = False):
    if(compute):
        x_train_mfcc = list(map(lambda x: get_mfcc_feature(x,20),X_train['wav_path']))
        x_test_mfcc = list(map(lambda x: get_mfcc_feature(x,20),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/mfcc20.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_mfcc,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_mfcc,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to mfcc pickle: ', e)
        return x_train_mfcc,y_train,x_test_mfcc, y_test
    else:
        try:
            with open('pickles/'+emotion+'/mfcc20.pickle','rb') as f:
                save = pickle.load(f)
                x_train_mfcc = save['train_dataset']
                y_train = save['train_labels']
                x_test_mfcc = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_mfcc,y_train,x_test_mfcc, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_rmse(emotion, X_train,X_test,y_train,y_test,compute = False):
    if(compute):
        x_train_rmse = list(map(lambda x: get_rmse(x),X_train['wav_path']))
        x_test_rmse = list(map(lambda x: get_rmse(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/rmse.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_rmse,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_rmse,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to rmse pickle: ', e)
        return x_train_rmse, y_train, x_test_rmse, y_test
    else:
        try:
            with open('pickles/'+emotion+'/rmse.pickle','rb') as f:
                save = pickle.load(f)
                x_train_rmse = save['train_dataset']
                y_train = save['train_labels']
                x_test_rmse = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_rmse, y_train, x_test_rmse, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_zcr(emotion,X_train,X_test,y_train,y_test,compute= False):
    if(compute):
        x_train_zcr = list(map(lambda x: get_zcr(x),X_train['wav_path']))
        x_test_zcr = list(map(lambda x: get_zcr(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/zcr.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_zcr,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_zcr,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to zcr pickle: ', e)
        return x_train_zcr, y_train, x_test_zcr, y_test
    else:
        try:
            with open('pickles/'+emotion+'/zcr.pickle','rb') as f:
                save = pickle.load(f)
                x_train_zcr = save['train_dataset']
                y_train = save['train_labels']
                x_test_zcr = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_zcr, y_train, x_test_zcr, y_test
        except Exception as e:
            print('Error at:', e)
    
def load_spectro(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train_spectro = list(map(lambda x: get_spectrogram(x),X_train['wav_path']))
        x_test_spectro = list(map(lambda x: get_spectrogram(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/spectrogram.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_spectro,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_spectro,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to spectrogram pickle: ', e)
        return x_train_spectro, y_train, x_test_spectro, y_test
    else:
        try:
            with open('pickles/'+emotion+'/spectrogram.pickle','rb') as f:
                save = pickle.load(f)
                x_train_spectro = save['train_dataset']
                y_train = save['train_labels']
                x_test_spectro = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_spectro, y_train, x_test_spectro, y_test
        except Exception as e:
            print('Error at:', e)
        
def load_melspec(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train_melspec = list(map(lambda x: get_melspectrogram(x),X_train['wav_path']))
        x_test_melspec = list(map(lambda x: get_melspectrogram(x),X_test['wav_path']))
        try:
            with open('pickles/anger/melspectrogram.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train_melspec,
                    'train_labels' : y_train,
                    'test_dataset' : x_test_melspec,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to melspectrogram pickle: ', e)
        return x_train_melspec, y_train, x_test_melspec, y_test
    else:
        try:
            with open('pickles/anger/melspectrogram.pickle','rb') as f:
                save = pickle.load(f)
                x_train_melspec = save['train_dataset']
                y_train = save['train_labels']
                x_test_melspec = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train_melspec, y_train, x_test_melspec, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_cqt(emotion,X_train,X_test,y_train,y_test,compute=False):
    if(compute):
        x_train = list(map(lambda x: get_cqt(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_cqt(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/cqt.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to cqt pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/cqt.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_chroma(emotion,X_train, X_test,y_train,y_test, compute=False):
    if(compute):
        x_train = list(map(lambda x: get_chroma(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_chroma(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/chroma_cqt.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to chroma cqt pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/chroma_cqt.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
            
def load_spectral_centroid(emotion,X_train,X_test,y_train,y_test, compute=False):
    if(compute):
        x_train = list(map(lambda x: get_spectral_centroid(x),X_train['wav_path']))
        x_test = list(map(lambda x: get_spectral_centroid(x),X_test['wav_path']))
        try:
            with open('pickles/'+emotion+'/spectral_centroid.pickle','wb') as f:
                save = {
                    'train_dataset' : x_train,
                    'train_labels' : y_train,
                    'test_dataset' : x_test,
                    'test_labels' : y_test
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to spectral centroid pickle: ', e)
        return x_train, y_train, x_test, y_test
    else:
        try:
            with open('pickles/'+emotion+'/spectral_centroid.pickle','rb') as f:
                save = pickle.load(f)
                x_train = save['train_dataset']
                y_train = save['train_labels']
                x_test = save['test_dataset']
                y_test = save['test_labels']
                del save
                return x_train, y_train, x_test, y_test
        except Exception as e:
            print('Error at:', e)
    

# Anger Binary Models

In [50]:
X_train,X_test,y_train,y_test = load_dataset(data,'anger')

In [53]:
x_train_mfcc,y_train,x_test_mfcc, y_test = load_mfcc('anger',X_train,X_test,y_train,y_test)
x_train_mfcc,x_test_mfcc = scale_features(x_train=x_train_mfcc,x_test=x_test_mfcc)
x_train_mfcc = pd.DataFrame(x_train_mfcc,columns=["mfcc_" + str(i+1) for i in range(20)])
x_test_mfcc = pd.DataFrame(x_test_mfcc,columns=["mfcc_"+str(i+1) for i in range(20)])

mfcc_models = compare_models(x_train_mfcc, y_train,x_test_mfcc, y_test)
print('MFCC Anger Model')
mfcc_models

MFCC Anger Model


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,74.89,0.792899,0.638095,0.707124
1,KNN,69.00,0.706215,0.595238,0.645995
2,Logistic Regression,75.57,0.768421,0.695238,0.730000
3,Random Forest,73.30,0.777108,0.614286,0.686170
4,Naive Bayes,71.95,0.731183,0.647619,0.686869
5,Perceptron,71.72,0.748538,0.609524,0.671916
6,Stochastic Gradient Decent,69.23,0.649194,0.766667,0.703057
7,Linear SVC,76.24,0.777778,0.700000,0.736842
8,Decision Tree,60.63,0.583333,0.600000,0.591549


In [58]:
x_train_rmse, y_train, x_test_rmse, y_test = load_rmse('anger',X_train,X_test,y_train,y_test)
x_train_rmse, x_test_rmse = pad_data(x_train_rmse, x_test_rmse)
x_train_rmse,x_test_rmse = scale_features(x_train=x_train_rmse,x_test=x_test_rmse)
n_features = x_train_rmse.shape[1]
# column_names = ["rmse_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

rmse_models = compare_models(x_train_rmse, y_train,x_test_rmse, y_test)
print('Root Mean Square Energy')
rmse_models
# x_train_rmse

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))
Root Mean Square Energy


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,64.93,0.823529,0.333333,0.474576
1,KNN,69.00,0.679803,0.657143,0.668281
2,Logistic Regression,67.19,0.677596,0.590476,0.631043
3,Random Forest,73.30,0.698276,0.771429,0.733032
4,Naive Bayes,66.52,0.803922,0.390476,0.525641
5,Perceptron,63.35,0.863636,0.271429,0.413043
6,Stochastic Gradient Decent,67.65,0.643777,0.714286,0.677201
7,Linear SVC,46.83,0.471264,0.976190,0.635659
8,Decision Tree,63.35,0.600000,0.685714,0.640000


In [56]:
x_train_zcr, y_train, x_test_zcr, y_test = load_zcr('anger',X_train,X_test,y_train,y_test)
x_train_zcr, x_test_zcr = pad_data(x_train_zcr, x_test_zcr)
# 
# n_features = x_train_zcr.shape[1]
column_names = ["zcr_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

# zcr_models = compare_models(x_train_zcr, y_train,x_test_zcr, y_test)
print('Zero Cross Rate')
# zcr_models
x_train_zcr

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))
Zero Cross Rate


,0,1,2,3,4,5,6,7,8,9,...,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374
0,0.019043,0.033203,0.044922,0.042480,0.042480,0.045898,0.051270,0.061523,0.067383,0.064941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.003906,0.006836,0.009766,0.015625,0.022461,0.030273,0.029297,0.027344,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.003906,0.004883,0.004883,0.006836,0.003906,0.008789,0.009766,0.007812,0.006836,0.000977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.014648,0.018555,0.019531,0.014648,0.005859,0.002441,0.005859,0.009766,0.016602,0.020508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
x_train_zcr,x_test_zcr = scale_features(x_train=x_train_zcr,x_test=x_test_zcr)
pd.DataFrame(x_train_zcr)

,0,1,2,3,4,5,6,7,8,9,...,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374
0,-0.785124,-0.749077,-0.738636,-0.755274,-0.754237,-0.735955,-0.713896,-0.665782,-0.611268,-0.533333,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.000000,-1.000000,-0.977273,-0.960619,-0.943503,-0.910112,-0.874659,-0.835544,-0.830986,-0.803509,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-0.955923,-0.963100,-0.971591,-0.960619,-0.977401,-0.949438,-0.945504,-0.957560,-0.960563,-0.992982,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,-0.834711,-0.859779,-0.886364,-0.915612,-0.966102,-0.985955,-0.967302,-0.946950,-0.904225,-0.852632,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [30]:
x_train_spectro, y_train, x_test_spectro, y_test = load_spectro('anger',X_train,X_test,y_train,y_test)
x_train_spectro,x_test_spectro = scale_features(x_train=x_train_spectro,x_test=x_test_spectro)
spectro_models = compare_models(x_train_spectro, y_train,x_test_spectro, y_test)
spectro_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,72.40,0.775000,0.590476,0.670270
1,KNN,72.17,0.743017,0.633333,0.683805
2,Logistic Regression,77.83,0.750000,0.800000,0.774194
3,Random Forest,72.17,0.727749,0.661905,0.693267
4,Naive Bayes,70.81,0.754717,0.571429,0.650407
5,Perceptron,73.98,0.698745,0.795238,0.743875
6,Stochastic Gradient Decent,73.98,0.733990,0.709524,0.721550
7,Linear SVC,77.15,0.753488,0.771429,0.762353
8,Decision Tree,62.67,0.622951,0.542857,0.580153


In [31]:
x_train_melspec, y_train, x_test_melspec, y_test = load_melspec('anger',X_train,X_test,y_train,y_test)
x_train_melspec,x_test_melspec = scale_features(x_train=x_train_melspec,x_test=x_test_melspec)
melspec_models = compare_models(x_train_melspec, y_train,x_test_melspec, y_test)
melspec_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,73.98,0.785000,0.685590,0.731935
1,KNN,70.59,0.712446,0.724891,0.718615
2,Logistic Regression,72.40,0.760976,0.681223,0.718894
3,Random Forest,72.85,0.771144,0.676856,0.720930
4,Naive Bayes,69.68,0.738693,0.641921,0.686916
5,Perceptron,72.62,0.781250,0.655022,0.712589
6,Stochastic Gradient Decent,73.53,0.727642,0.781659,0.753684
7,Linear SVC,73.98,0.776699,0.698690,0.735632
8,Decision Tree,63.80,0.641975,0.681223,0.661017


In [33]:
x_train_cqt, y_train, x_test_cqt, y_test = load_cqt('anger',X_train,X_test,y_train,y_test)
x_train_cqt,x_test_cqt = scale_features(x_train=x_train_cqt,x_test=x_test_cqt)
cqt_models = compare_models(x_train_cqt, y_train,x_test_cqt, y_test)
cqt_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,70.36,0.680365,0.709524,0.694639
1,KNN,70.81,0.673820,0.747619,0.708804
2,Logistic Regression,71.49,0.705882,0.685714,0.695652
3,Random Forest,73.30,0.716981,0.723810,0.720379
4,Naive Bayes,69.23,0.692708,0.633333,0.661692
5,Perceptron,67.19,0.638298,0.714286,0.674157
6,Stochastic Gradient Decent,70.36,0.758170,0.552381,0.639118
7,Linear SVC,71.27,0.687783,0.723810,0.705336
8,Decision Tree,57.69,0.550218,0.600000,0.574032


In [34]:
x_train_chroma, y_train, x_test_chroma, y_test = load_chroma('anger',X_train,X_test,y_train,y_test)
x_train_chroma,x_test_chroma = scale_features(x_train=x_train_chroma,x_test=x_test_chroma)
chroma_models = compare_models(x_train_chroma, y_train,x_test_chroma, y_test)
chroma_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,62.22,0.589212,0.676190,0.629712
1,KNN,54.98,0.524444,0.561905,0.542529
2,Logistic Regression,63.35,0.604348,0.661905,0.631818
3,Random Forest,60.18,0.594444,0.509524,0.548718
4,Naive Bayes,61.31,0.587444,0.623810,0.605081
5,Perceptron,58.14,0.573964,0.461905,0.511873
6,Stochastic Gradient Decent,56.79,0.527221,0.876190,0.658318
7,Linear SVC,62.22,0.605911,0.585714,0.595642
8,Decision Tree,51.58,0.489474,0.442857,0.465000


In [35]:
x_train_specentr, y_train, x_test_specentr, y_test = load_spectral_centroid('anger',X_train,X_test,y_train,y_test)
x_train_specentr, x_test_specentr = pad_data(x_train_specentr, x_test_specentr)
x_train_specentr,x_test_specentr = scale_features(x_train=x_train_specentr,x_test=x_test_specentr)
n_features = x_train_specentr.shape[1]
# column_names = ["spectralCent_" + str(i+1) for i in range(n_features)]
# x_train_specentr = pd.DataFrame(x_train_specentr,columns=column_names)
# x_test_specentr = pd.DataFrame(x_test_specentr,columns=column_names)

specentr_models = compare_models(x_train_specentr, y_train,x_test_specentr, y_test)
specentr_models

((1764, 1375), (442, 1255))
((1764, 1375), (442, 1375))


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,61.76,0.594470,0.614286,0.604215
1,KNN,57.24,0.558659,0.476190,0.514139
2,Logistic Regression,57.92,0.561856,0.519048,0.539604
3,Random Forest,66.06,0.633929,0.676190,0.654378
4,Naive Bayes,47.74,0.475973,0.990476,0.642968
5,Perceptron,52.94,0.625000,0.023810,0.045872
6,Stochastic Gradient Decent,49.10,0.482100,0.961905,0.642289
7,Linear SVC,54.30,0.511834,0.823810,0.631387
8,Decision Tree,54.30,0.515504,0.633333,0.568376


In [59]:
x_train_combine1 = combine_features(x_train_mfcc,x_train_rmse)
x_test_combine1 = combine_features(x_test_mfcc,x_test_rmse)
combine1_models = compare_models(x_train_combine1,y_train,x_test_combine1,y_test)
print('MFCC + RMSE')
combine1_models

TypeError: cannot concatenate object of type "<type 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [274]:
x_train_combine2 = combine_features(x_train_combine1,x_train_zcr)
x_test_combine2 = combine_features(x_test_combine1,x_test_zcr)
combine2_models = compare_models(x_train_combine2,y_train,x_test_combine2,y_test)
print('MFCC + RMSE + ZCR')
combine2_models

MFCC + RMSE + ZCR


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,52.71,0.547511,0.526087,0.536585
1,KNN,50.45,0.526316,0.478261,0.501139
2,Logistic Regression,48.42,0.505155,0.426087,0.462264
3,Random Forest,51.58,0.538462,0.486957,0.511416
4,Naive Bayes,52.49,0.524631,0.926087,0.669811
5,Perceptron,52.04,0.520362,1.000000,0.684524
6,Stochastic Gradient Decent,52.04,0.520362,1.000000,0.684524
7,Linear SVC,47.96,0.000000,0.000000,NaN
8,Decision Tree,50.90,0.526531,0.560870,0.543158


In [297]:
# x_train_melspec = pd.DataFrame(x_train_melspec)
# x_test_melspec = pd.DataFrame(x_test_melspec)
x_train_combine3 = combine_features(x_train_combine2,x_train_melspec)
x_test_combine3 = combine_features(x_test_combine2,x_test_melspec)
combine3_models = compare_models(x_train_combine3,y_train,x_test_combine3,y_test)
print('MFCC + RMSE + ZCR + MELSPEC')
combine3_models

MFCC + RMSE + ZCR + MELSPEC


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,74.21,0.733010,0.719048,0.725962
1,KNN,67.87,0.663462,0.657143,0.660287
2,Logistic Regression,77.38,0.772277,0.742857,0.757282
3,Random Forest,75.79,0.746411,0.742857,0.744630
4,Naive Bayes,60.86,0.849057,0.214286,0.342205
5,Perceptron,60.41,0.872340,0.195238,0.319066
6,Stochastic Gradient Decent,68.33,0.836538,0.414286,0.554140
7,Linear SVC,61.99,0.556757,0.980952,0.710345
8,Decision Tree,62.67,0.605634,0.614286,0.609929


In [271]:
# X_test[['text','emotion']].iloc[fp].head(10)
# play_audio(X_test.iloc[fp]['wav_path'].values[3])

# Sadness Binary Models

In [298]:
X_train,X_test,y_train,y_test = load_dataset(data,'sadness',True)

In [301]:
x_train_mfcc,y_train,x_test_mfcc, y_test = load_mfcc('sadness',X_train,X_test,y_train,y_test)
x_train_mfcc = pd.DataFrame(x_train_mfcc,columns=["mfcc_" + str(i+1) for i in range(20)])
x_test_mfcc = pd.DataFrame(x_test_mfcc,columns=["mfcc_"+str(i+1) for i in range(20)])

mfcc_models = compare_models(x_train_mfcc, y_train,x_test_mfcc, y_test)
print('MFCC Sadness Model')
mfcc_models

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_M016.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro03/Ses05M_impro03_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_F004.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro05/Ses04F_impro05_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro06/Ses05M_impro06_F007.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_F013.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_1/Ses04M_script02_1_F008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script01_1b/Ses05M_script01_1b_M020.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_F000.wav
Finished d

Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_F066.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_F038.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script03_2/Ses01F_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro08/Ses04F_impro08_M000.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script03_2/Ses05M_script03_2_M022.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script03_2/Ses04M_script03_2_M035.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_impro05/Ses01M_impro05_M025.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F017.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_M034.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro01/Ses04M_impro01_M018.wa

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M021.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F001.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro07/Ses02M_impro07_F003.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_1/Ses03M_script01_1_M041.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_3/Ses03F_script01_3_M038.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script02_2/Ses04M_script02_2_F022.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F024.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_M021.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_impro07/Ses04F_impro07_M022.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro02/Ses03M_impro02_M

Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro06/Ses01F_impro06_F003.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_3/Ses01M_script01_3_F012.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro05/Ses05F_impro05_M032.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05a/Ses03M_impro05a_M023.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_2/Ses01M_script02_2_F008.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_script02_2/Ses02M_script02_2_F006.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro07/Ses02F_impro07_F004.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_M033.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_script02_2/Ses05M_script02_2_F009.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script02_1/Ses05F_script02_1_M

Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro08/Ses05F_impro08_F019.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_impro06/Ses05F_impro06_F027.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_F020.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro06/Ses02M_impro06_M008.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro05/Ses05M_impro05_M004.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro04/Ses02M_impro04_M013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro04/Ses03M_impro04_M009.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impro08a_M000.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F000.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_F038.wav
Finished data/

...

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script01_1/Ses03F_script01_1_F013.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_script03_2/Ses02F_script03_2_M036.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro05b/Ses03M_impro05b_M022.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_2/Ses05F_script01_2_M014.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script02_1/Ses01M_script02_1_F014.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_M001.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script03_2/Ses03M_script03_2_M011.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03F_script03_2/Ses03F_script03_2_F014.wav
Finished data/IEMOCAP_full_release/Session5/sentences/wav/Ses05F_script01_1/Ses05F_script01_1_F034.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_1/S

Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_script01_2/Ses03M_script01_2_F001.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_script01_2/Ses04M_script01_2_F015.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2/Ses01M_script03_2_F036.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script01_2/Ses01M_script01_2_F012.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F029.wav
Finished data/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script01_2/Ses01F_script01_2_F007.wav
Finished data/IEMOCAP_full_release/Session2/sentences/wav/Ses02M_impro08/Ses02M_impro08_F010.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_2/Ses04F_script02_2_F011.wav
Finished data/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script03_2/Ses04F_script03_2_F013.wav
Finished data/IEMOCAP_full_release/Session3/sentences/wav/Ses03M_impro08a/Ses03M_impr

/usr/local/Cellar/ipython@5/5.7.0/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars


MFCC Sadness Model


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,48.19,0.000000,0.000000,NaN
1,KNN,71.72,0.742991,0.694323,0.717833
2,Logistic Regression,73.30,0.763033,0.703057,0.731818
3,Random Forest,74.43,0.763636,0.733624,0.748330
4,Naive Bayes,70.59,0.726027,0.694323,0.709821
5,Perceptron,69.00,0.794872,0.541485,0.644156
6,Stochastic Gradient Decent,51.81,0.518100,1.000000,0.682563
7,Linear SVC,53.39,0.960000,0.104803,0.188976
8,Decision Tree,64.71,0.652720,0.681223,0.666667


In [18]:
x_train_rmse, y_train, x_test_rmse, y_test = load_rmse('sadness',X_train,X_test,y_train,y_test)
    
x_train_rmse, x_test_rmse = pad_data(x_train_rmse, x_test_rmse)
n_features = x_train_rmse.shape[1]
# column_names = ["rmse_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

rmse_models = compare_models(x_train_rmse, y_train,x_test_rmse, y_test)
rmse_models

((1764, 997), (442, 1154))
((1764, 1154), (442, 1154))


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,50.00,1.000000,0.034934,0.067511
1,KNN,64.48,0.768657,0.449782,0.567493
2,Logistic Regression,65.61,0.758389,0.493450,0.597884
3,Random Forest,71.27,0.742857,0.681223,0.710706
4,Naive Bayes,54.30,0.846154,0.144105,0.246269
5,Perceptron,65.16,0.741935,0.502183,0.598958
6,Stochastic Gradient Decent,68.10,0.676000,0.737991,0.705637
7,Linear SVC,64.48,0.757143,0.462882,0.574526
8,Decision Tree,61.54,0.649746,0.558952,0.600939


In [19]:
x_train_zcr, y_train, x_test_zcr, y_test = load_zcr('sadness',X_train,X_test,y_train,y_test)

x_train_zcr, x_test_zcr = pad_data(x_train_zcr, x_test_zcr)
n_features = x_train_zcr.shape[1]
column_names = ["zcr_" + str(i+1) for i in range(n_features)]
# x_train_rmse = pd.DataFrame(x_train_rmse,columns=column_names)
# x_test_rmse = pd.DataFrame(x_test_rmse,columns=column_names)

zcr_models = compare_models(x_train_zcr, y_train,x_test_zcr, y_test)
zcr_models

((1764, 997), (442, 1154))
((1764, 1154), (442, 1154))


/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/Cellar/ipython@5/5.7.0/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in double_scalars


,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,48.19,0.000000,0.000000,NaN
1,KNN,62.90,0.699387,0.497817,0.581633
2,Logistic Regression,67.65,0.712871,0.628821,0.668213
3,Random Forest,74.21,0.748918,0.755459,0.752174
4,Naive Bayes,51.58,0.826087,0.082969,0.150794
5,Perceptron,64.25,0.622837,0.786026,0.694981
6,Stochastic Gradient Decent,67.19,0.712121,0.615721,0.660422
7,Linear SVC,66.06,0.700508,0.602620,0.647887
8,Decision Tree,61.31,0.628319,0.620087,0.624176


In [20]:
x_train_spectro, y_train, x_test_spectro, y_test = load_spectro('sadness',X_train,X_test,y_train,y_test)
spectro_models = compare_models(x_train_spectro, y_train,x_test_spectro, y_test)
spectro_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,67.87,0.682008,0.711790,0.696581
1,KNN,68.55,0.725000,0.633188,0.675991
2,Logistic Regression,69.23,0.722488,0.659389,0.689498
3,Random Forest,72.62,0.762136,0.685590,0.721839
4,Naive Bayes,69.46,0.737374,0.637555,0.683841
5,Perceptron,51.81,0.518100,1.000000,0.682563
6,Stochastic Gradient Decent,67.19,0.828125,0.462882,0.593838
7,Linear SVC,55.88,1.000000,0.148472,0.258555
8,Decision Tree,68.10,0.691304,0.694323,0.692810


In [21]:
x_train_melspec, y_train, x_test_melspec, y_test = load_melspec('sadness',X_train,X_test,y_train,y_test)
melspec_models = compare_models(x_train_melspec, y_train,x_test_melspec, y_test)
melspec_models

,Model,Accuracy,Precision,Recall,F1
0,Support Vector Machines,67.19,0.636364,0.855895,0.729981
1,KNN,69.00,0.712963,0.672489,0.692135
2,Logistic Regression,71.49,0.751220,0.672489,0.709677
3,Random Forest,73.76,0.772947,0.698690,0.733945
4,Naive Bayes,68.78,0.735751,0.620087,0.672986
5,Perceptron,69.23,0.839416,0.502183,0.628415
6,Stochastic Gradient Decent,51.81,0.518100,1.000000,0.682563
7,Linear SVC,64.25,0.898876,0.349345,0.503145
8,Decision Tree,67.87,0.702326,0.659389,0.680180
